In [ ]:
from vllm import LLM, SamplingParams
import torch 
import pandas as pd
from torchmetrics.text import ROUGEScore


In [ ]:
rouge = ROUGEScore()

In [ ]:
sampling_params = SamplingParams(temperature=0.0, top_p=0.95, max_tokens=512)

In [ ]:
batch_size=64

In [ ]:
rev_df = pd.read_csv("../../../data/labelled/reviews/splits/test.csv")

In [ ]:
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.1", dtype=torch.float16, gpu_memory_utilization=0.95, enforce_eager=True)

WARNING 04-14 04:27:13 config.py:748] Casting torch.bfloat16 to torch.float16.
INFO 04-14 04:27:13 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='mistralai/Mistral-7B-Instruct-v0.1', tokenizer='mistralai/Mistral-7B-Instruct-v0.1', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=True, kv_cache_dtype=auto, device_config=cuda, seed=0)
INFO 04-14 04:27:13 selector.py:45] Cannot use FlashAttention because the package is not found. Please install it for better performance.
INFO 04-14 04:27:13 selector.py:21] Using XFormers backend.
INFO 04-14 04:27:15 weight_utils.py:177] Using model weights format ['*.safetensors']
INFO 04-14 04:27:21 model_runner.py:104] Loading model weights took 13.4966 GB
INFO 04-14 04:27:23 gpu_executor.py:94] # GPU blocks: 29400, # CPU block

In [ ]:
preds = []
for idx in range(0, len(rev_df), batch_size):
    if (idx)/(batch_size*100) == 0: print(f"processed {idx+1} rows")
    batch = rev_df[idx:idx+batch_size]
    descs = batch.input_reviews.to_list()
    labels = batch.label_reviews.to_list()
    input_batch = ["<s>[INST] "+desc+" [/INST]" for desc in descs]
    outputs = llm.generate(input_batch, sampling_params, use_tqdm=False)
    preds.extend([out.outputs[0].text.strip() for out in outputs])    

In [ ]:
rev_df['mistral'] = preds

In [ ]:
rev_df.to_csv("mistral_results_reviews.csv", index=False)

In [ ]:
rev_df.head(2)

,input_reviews,label_reviews,mistral
0,Below are the reviews of a product: \n- I purc...,pros:\n- Reduces glare\n- Very impressive\n- W...,Pros:\n\n* High-quality multi-coated glass\n* ...
1,Below are the reviews of a product: \n- Well w...,pros:\n- Huge capacity\n- Beautiful design\n- ...,The Astro Pro 20000mAh 4-Port Aluminum Portabl...


In [ ]:
rouge_scores = {"rouge1":[], "rouge2":[], "rougeL": []}
for pred, target in zip(rev_df.label_reviews.to_list(), rev_df.mistral.to_list()):
    rouge_all = rouge(pred, target)
    rouge_scores['rouge1'].append(rouge_all['rouge1_fmeasure'].item())
    rouge_scores['rouge2'].append(rouge_all['rouge2_fmeasure'].item())
    rouge_scores['rougeL'].append(rouge_all['rougeL_fmeasure'].item())

In [ ]:
rev_df['rouge1'] = rouge_scores['rouge1']
rev_df['rouge2'] = rouge_scores['rouge2']
rev_df['rougeL'] = rouge_scores['rougeL']

In [ ]:
rev_df.to_csv("mistral_results_reviews.csv", index=False)

In [ ]:
scores = {"rouge1": rev_df.rouge1.mean(), "rouge2": rev_df.rouge2.mean(), "rougeL": rev_df.rougeL.mean()}

In [ ]:
scores

{'rouge1': 0.3995208429379389,
 'rouge2': 0.157890436598612,
 'rougeL': 0.27184627608354206}